## Accelerating inference with sparsity using NVIDIA Ampere architecure and NVIDIA TensorRT - NVIDIA Blog 2021

Quando si sviluppano reti neurali è di utilità pensare a come rendere tali reti più veloci e più piccole nel senso della dimensione spaziale. Una rete più efficiente potrebbe fare previsioni migliori in meno tempo ed essere più adatta al deployment su ambienti vincolati cioè dispositivi con risorse limitate. La sparsificazione è una delle tecniche che potrebbe portare ad una riduzione del tempo di inferenza e ad una riduzione delle dimensioni della rete. Se i tensori dei pesi sono sparsificati cioè composti da molti valori nulli, questi non vanno considerati nei calcoli e non è necessario memorizzarli con conseguenti benefici temporali e spaziali. Tuttavia vi sono alcune problematiche:
+ Accelerazione: se i valori nulli, all'interno dei tensori, sono distribuiti casualmente, senza una struttura logica particolare (in tal caso si parlerà di sparsificazione non strutturata), non sarà possibile sfruttare tutte quelle operazioni fra matrici e vettori supportate direttamente dall'hardware.
+ Accuratezza: il processo di sparsificazione di una rete può portare ad un miglioramento del tempo di inferenza ma allo stesso tempo produce un peggioramento nell'accuratezza della rete stessa.
+ Workflow: la maggior parte delle ricerche correnti mostrano che dato modello A è possibile raggiungere un grado di sparsificazione X. Il problema sorge quando si tenta di applicare il grado di sparsificazione X ad un altro modello B. La sparsificazione potrebbe non funzionare a causa della diversa architettura della rete, del task da risolvere o qualsiasi altro iperparametro.

L'architettura Ampere per GPU NVIDIA indirizza queste problematiche. La libreria TensorRT, dalla versione 8.0, introduce supporto all'utilizzo degli Sparse Tensor Cores disponibili sulle architetture Ampere. (Cosa è e come è fatto e come funziona uno Sparse Tensor Core?). Lo sfruttamento degli Sparse Tensor Cores consente di ignorare i calcoli non necessari (ovvero quelli che coinvolgono valori dei pesi nulli) portando ad un aumento anche superiore al 30% del rapporto prestazioni/potenza(W) rispetto alle reti neurali dense

Gli Sparse Tensor Cores sono in grado di accelerare calcoli che coinvolgano tensori sparsificati secondo lo schema strutturato 2:4 cioè per ogni blocco contiguo di 4 valori due di essi devono essere nulli. Questo porta ad una sparsità del 50%. Un tensore conforme a questo pattern è facile da comprimere.

![matrix 2:4](img\matrix_2_4.png)

La matrice compressa, come rappresentato nella figura sopra, ha R righe e C/2 colonne ed è accompagnata da una matrice Rx(C/2) composta da indici rappresentati usando 2 bit ciascuno. Gli Sparse Tensor Cores sono in grado di accelerare i calcoli relativi a tensori aventi questo formato compresso considerando solo i valori non nulli. Utilizzano la matrice di indice (detta anche matrice dei metadati) per selezionare soltanto i valori che non siano nulli. Per sparsità al 50% possono eseguire i calcoli in metà tempo.

![](img\sparseTensorPerformance.png)

Come mostrato dalla tabella, la sparsità porta ad una maggiore velocità delle operazioni indipendentemente dalla tipologia di dato.

La sparsità 2:4 consente di mantenere l'accuratezza rispetto al modello denso. Un modello molto veloce sarebbe del tutto inutile se non fosse accurato. E' stato quindi sviluppato un processo che consente di generare una sparsità strutturata di tipo 2:4 su di una rete neurale e che porti ad una accuratezza uguale a quella del modello denso:
+ si parte da una rete densa addestrata che abbia raggiunto un ottimo livello di accuratezza
+ si passa ad una fase di potatura dei pesi che porti a soddisfare il pattern di sparsità strutturato di tipo 2:4. Ogni quattro elementi ne rimuoviamo esattamente due.
+ poi procediamo al riaddestramento della rete
Nella fase di pruning bisogna decidere quali valori mantenere e quali valori porre a zero. Serve dunque un criterio. Quello più semplice è basato sul valore assoluto dei pesi (weight magnitude): poniamo a zero i valori che sono più prossimi ad esso. A questo punto sorge un problema. Impostando a zero la metà dei pesi di una rete neurale si avranno ripercussioni sull'accuratezza in fase di inferenza. Ecco giustificato l'ultimo passo del processo precedente che consiste in una ulteriore fase di addestramento. Questa servirà ad aggiornare i pesi rimasti quel tanto che sia sufficiente a recuperare l'accuratezza perduta successivamente alla fase di pruning.

![accuracy_2_4_table](img\accuracy2_4.png)

### Sommario

La sparsificazione è una tecnica popolare per comprimere reti neurali. Tuttavia, un grado di sparsità elevato, non consente di ottenere quei guadagni di prestazioni ed accuratezza promessi dalla ricerca. I ricercatori NVIDIA, hanno ideato il pattern di sparsificazione strutturata di tipo 2:4 ed implementato un supporto ad esso direttamente negli Sparse Tensor Cores dell'architettura NVIDIA Ampere. Questo consente di generare reti neurali sparsificate senza alcuna perdita di accuratezza rispetto al modello denso e successivamente TensorRT si occuperà dell'accelerazione in termini computazionali. Il risultato sarà un rete neurale più piccola, compressa, con necessità di memoria inferiori, più veloce in fase di inferenza grazie alle ottimizzazioni incapsulate direttamente all'interno dell'hardware ed accurata tanto quanto la rete neurale densa da cui essa deriva.